In [2]:
import csv, sys, os, math
import pandas as pd
import numpy as np
from numpy import argmax
import random
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score,roc_curve,auc  
import matplotlib.pyplot as plt
import matplotlib
from matplotlib import pyplot
from functools import reduce
from pathlib import Path
from sklearn.model_selection import StratifiedKFold
workdir = os.path.abspath('.')

In [3]:
import arfs
from arfs.feature_selection import GrootCV, Leshy
from arfs.utils import load_data
from arfs.benchmark import highlight_tick

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [4]:
exp_data = pd.read_csv('%s/BRCA.sampleMap_HiSeqV2'%workdir,sep='\t')
gene_name = exp_data['sample']
exp_data = exp_data.drop(labels = 'sample',axis=1).transpose()
exp_data.columns = gene_name
ylable= pd.DataFrame(pd.DataFrame(exp_data.index)[0].str.split('-').str[-1])
exp_data['ylable'] = np.where(ylable[0].str.contains("0"), 1, 0)

x = exp_data.iloc[:,0:20531]
x = x.drop(columns = 'ylable')
y = exp_data.iloc[:,-1].astype(int)

# Fastshap enable

In [5]:
feat_selector = GrootCV(
    objective="rmse",
    cutoff=1,
    n_folds=3,
    n_iter=3,
    silent=True,
    fastshap=True,
    n_jobs=0,
    lgbm_params={"device": "cpu"},
)

In [6]:
importance_list = []
x_train,x_test,y_train,y_test = train_test_split(x, y, 
                                                 test_size = 0.2,
                                                 random_state = 32, 
                                                 stratify=y)
feat_selector.fit(x_train, y_train, sample_weight=None)
importance_list.append(feat_selector.ranking_absolutes_)

Repeated k-fold:   0%|          | 0/9 [00:00<?, ?it/s]

GrootCV(fastshap=True,
        lgbm_params={'device': 'cpu', 'num_threads': 0, 'objective': 'rmse',
                     'verbosity': -1},
        n_folds=3, n_iter=3, objective='rmse')

In [19]:
len(feat_selector.ranking_absolutes_)

20530

In [21]:
name=['GrootCV.fastshap']
name2=range(1,len(feat_selector.ranking_absolutes_)+1)
df_importance=pd.DataFrame(columns=name,index=name2,data=feat_selector.ranking_absolutes_)
print(df_importance)

      GrootCV.fastshap
1              ADAMTS5
2                MMP11
3                 FIGF
4             PPP1R12B
5              CD300LG
...                ...
20526     LOC100169752
20527            FITM1
20528           IFNA21
20529          RPS10P7
20530             SELS

[20530 rows x 1 columns]


In [22]:
df_importance.to_csv('GrootCV.fastshap特征排序结果.csv')

# Fastshap disable

In [23]:
feat_selector1 = GrootCV(
    objective="rmse",
    cutoff=1,
    n_folds=3,
    n_iter=3,
    silent=True,
    fastshap=False,
    n_jobs=0,
    lgbm_params={"device": "cpu"},
)
feat_selector1.fit(x_train, y_train, sample_weight=None)

Repeated k-fold:   0%|          | 0/9 [00:00<?, ?it/s]

GrootCV(lgbm_params={'device': 'cpu', 'num_threads': 0, 'objective': 'rmse',
                     'verbosity': -1},
        n_folds=3, n_iter=3, objective='rmse')

In [24]:
feat_selector.get_feature_names_out()

array(['PAMR1', 'ARHGAP20', 'SPRY2', 'SDPR', 'COL10A1', 'TSLP', 'FIGF',
       'MME', 'PPP1R12B', 'CD300LG', 'LYVE1', 'MMP11', 'LMOD1', 'HSD17B6',
       'SCARA5', 'TMEM220', 'SCN4B', 'ADAMTS5', 'MAMDC2', 'TNXB', 'SYNM'],
      dtype=object)

In [28]:
feat_selector1.get_feature_names_out()

array(['PAMR1', 'ARHGAP20', 'SPRY2', 'SDPR', 'COL10A1', 'TSLP', 'FIGF',
       'MME', 'PPP1R12B', 'C1QTNF9', 'CD300LG', 'LYVE1', 'MMP11', 'LMOD1',
       'HSD17B6', 'TMEM220', 'SCN4B', 'ADAMTS5', 'MAMDC2', 'TNXB', 'SYNM'],
      dtype=object)

In [25]:
feat_selector.ranking_

array([1, 1, 1, ..., 1, 1, 1])

In [27]:
feat_selector1.ranking_absolutes_

['ADAMTS5',
 'MMP11',
 'FIGF',
 'PPP1R12B',
 'CD300LG',
 'TMEM220',
 'LMOD1',
 'SPRY2',
 'COL10A1',
 'MME',
 'SCN4B',
 'PAMR1',
 'SDPR',
 'TSLP',
 'HSD17B6',
 'MAMDC2',
 'SYNM',
 'ARHGAP20',
 'LYVE1',
 'TNXB',
 'C1QTNF9',
 'FXYD1',
 'SCARA5',
 'BGN',
 'CXCL12',
 'CLEC5A',
 'CEP68',
 'LOC399959',
 'BMP8A',
 'LNP1',
 'SLC7A14',
 'C1QTNF9B',
 'SLCO1A2',
 'ANXA1',
 'MMP13',
 'GIPC2',
 'LRRTM2',
 'CAV1',
 'LRRC2',
 'ATP1A2',
 'SLC17A7',
 'C3orf64',
 'CXCL2',
 'HIF3A',
 'SMYD1',
 'HBA2',
 'MYLK',
 'TMEM63B',
 'CLDN5',
 'NCAM1',
 'NKAPL',
 'PPAP2B',
 'FGF1',
 'TCEAL7',
 'KIAA0101',
 'TMEM183A',
 'HPSE2',
 'PRNP',
 'CX3CL1',
 'HOXA5',
 'CASQ2',
 'SVEP1',
 'ALDH1A2',
 'PAQR4',
 'LOC285419',
 'PPP4C',
 'DMD',
 'SEMA6D',
 'USP44',
 'LRRC3B',
 'N4BP2L1',
 'C18orf34',
 'JAM3',
 'COL11A1',
 'TSPAN7',
 'CHRDL1',
 'SCN4A',
 'RUSC1',
 'CAV2',
 'SGCZ',
 'FAM162B',
 'CPA1',
 'ART4',
 'EFHA2',
 'WHAMML2',
 'NIPSNAP3B',
 'TTC28',
 'NECAB1',
 'LOC572558',
 'PTN',
 'CTHRC1',
 'ZNF658',
 'CRB2',
 'DUSP6',
 'A

In [29]:
name1=['GrootCV']
name3=range(1,len(feat_selector.ranking_absolutes_)+1)
df_importance1=pd.DataFrame(columns=name1,index=name3,data=feat_selector1.ranking_absolutes_)
print(df_importance1)

        GrootCV
1       ADAMTS5
2         MMP11
3          FIGF
4      PPP1R12B
5       CD300LG
...         ...
20526    ZNF513
20527    ZNF511
20528    OR5AR1
20529      HNMT
20530      SELS

[20530 rows x 1 columns]


In [30]:
df_importance1.to_csv('GrootCV特征排序结果.csv')